<a href="https://colab.research.google.com/github/djfLtC0dr/python-playground/blob/main/DASC522/djfDASC522_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

a. Break your code into logical chunks => SOP imports

In [17]:
# importing libraries
import statsmodels.api as sm
import pandas as pd 

# loading the training dataset 
data = pd.read_csv('/home/LRS_Pre_Assessment_trimmed_rank.csv')
print(data)

    Flight        Rank  Age Gender  ORS_total  PTSD_score  Sleep_score  \
0        D  JrEnlisted   40      M        9.8           5            8   
1        D  JrEnlisted   22      M        5.0          22           10   
2        D  JrEnlisted   20      M        9.6           1            3   
3        M  JrEnlisted   21      F        5.0          12           13   
4        D  JrEnlisted   27      M        8.3           0            6   
..     ...         ...  ...    ...        ...         ...          ...   
152      M  SrEnlisted   39      F        3.3          28           12   
153      M  JrEnlisted   21      F        4.7          13            7   
154      M  JrEnlisted   23      F        8.7           0            6   
155      M  JrEnlisted   20      M        5.7          12            4   
156     CC  SrEnlisted   34      M        9.0           8            3   

     survey_Burnout  survey_FitSat   BMI  BodyFatPerc  MusclePerc  \
0                 2              3  27.9  

b. Drop the “flight” column and one-hot-encode the “rank” & “gender” *columns*

In [18]:
data.drop(columns=['Flight'], axis=1, inplace=True)
data_encoded = pd.get_dummies(data, columns = ['Rank','Gender'])

c. Create a Data Understanding table & include 3 Data Understanding visualizations

In [19]:
data_encoded.describe()

,Age,ORS_total,PTSD_score,Sleep_score,survey_Burnout,survey_FitSat,BMI,BodyFatPerc,MusclePerc,APFT_1_is_pass,Rank_JrEnlisted,Rank_SrEnlisted,Rank_officer,Gender_F,Gender_M
count,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000,157.000000
mean,27.942675,7.620382,9.312102,7.305732,2.140127,3.331210,27.110191,0.294873,0.327968,0.707006,0.452229,0.509554,0.038217,0.242038,0.757962
std,6.164666,1.955621,11.795182,3.939710,0.812235,0.803746,4.093791,0.086974,0.060547,0.456592,0.499305,0.501508,0.192332,0.429688,0.429688
min,19.000000,0.200000,0.000000,0.000000,1.000000,1.000000,18.400000,0.014700,0.036500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,6.700000,1.000000,4.000000,2.000000,3.000000,24.400000,0.248000,0.289000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,26.000000,8.000000,5.000000,7.000000,2.000000,3.000000,27.100000,0.296000,0.334000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,33.000000,9.000000,12.000000,10.000000,2.000000,4.000000,29.700000,0.346000,0.366000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,45.000000,10.000000,68.000000,22.000000,5.000000,5.000000,38.300000,0.495000,0.473000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
